In [30]:
import io
import pandas as pd
import requests

def ades_GetPointEau(numéro_département: int=None, nb_réseau: int=None):
    """Retourne liste de dictionnaires"""
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEau'
    req_data = {
        'type': '1',
        'filter': f'departement={numéro_département}&nbReseau={nb_réseau}',
        # 'rows': 500,
        # 'start': '0',
        # 'statut_qualif': '0'
    }
    response_GetPointEau = requests.post(url=url, data=req_data)
    return response_GetPointEau.json()['pointEaux']


def formater_données_ades_GetPointEau(point_eau_liste: list=None):
    """Formater données retournées par fonction ades_GetPointEau"""
    point_eau_liste_formatée = []
    for p in point_eau_liste:
        dic = {}
        dic['coordinates0'] = p['Coordinates'][0]
        dic['coordinates1'] = p['Coordinates'][1]
        dic['coord_com'] = p['Properties']['coord_com']
        dic['coord_ref0'] = p['Properties']['coord_ref'][0]
        dic['coord_ref1'] = p['Properties']['coord_ref'][1]
        dic['epsg_ref'] = p['Properties']['epsg_ref']
        dic['exist_mesures'] = p['Properties']['exist_mesures']
        if ' ' in p['Properties']['label']:
            dic['Identifiant BSS'] = p['Properties']['label'].split(' ')[0]
            dic['Ancien code national BSS'] = p['Properties']['label'].split(' ')[1].replace('(', '').replace(')', '')
        else:
            dic['Identifiant BSS'] = p['Properties']['label']
            dic['Ancien code national BSS'] = 'N/D'
        dic['position_captage'] = p['Properties']['position_captage']
        dic['type_pt_eau'] = p['Properties']['type_pt_eau']
        point_eau_liste_formatée.append(dic)
        return point_eau_liste_formatée


def ades_GetPointEauIPSDataCSV(numéro_département=None, nombre_points=None):
    """Retourne le contenu d'un fichier CSV sous forme d'octets"""
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEauIPSDataCSV'
    query_params = {
        # 'rows': -1,
        # 'start': 0,
        'filters': f'departement={numéro_département}',
        'nombre_points': nombre_points,
        'typeRecherche': 1,
        # 'date_debut': '',
        # 'date_fin': '',
        # 'moisAnnee': ''
    }
    response_GetPointEauIPSDataCSV = requests.get(url=url, params=query_params)
    content_as_bytes = response_GetPointEauIPSDataCSV.content
    return content_as_bytes


numéro_département = 66
point_eau_liste = ades_GetPointEau(numéro_département, nb_réseau=0)
point_eau_liste_formatée = formater_données_ades_GetPointEau(point_eau_liste)
del point_eau_liste
point_eau_ips_data_csv_bytes = ades_GetPointEauIPSDataCSV(numéro_département, 
                                                          nombre_points=len(point_eau_liste_formatée))
df_point_eau = pd.merge(left=pd.DataFrame(point_eau_liste_formatée), 
                        right=pd.read_csv(io.BytesIO(point_eau_ips_data_csv_bytes), sep=';'),
                        on=['Identifiant BSS', 'Ancien code national BSS'])
df_point_eau
# df_point_eau[df_point_eau['Ancien code national BSS'] == '10795X0001/S']

,coordinates0,coordinates1,coord_com,coord_ref0,coord_ref1,epsg_ref,exist_mesures,Identifiant BSS,Ancien code national BSS,position_captage,type_pt_eau,Type de suivi,Commune,Département,Nb de mesures,Date de début,Date de fin,Nb de réseaux
0,2.033499,42.489388,N,620427.0518682573,6154881.538714178,2154,None,BSS002MSAL,10947X0084/F,None,1,Piézomètre,Font-Romeu-Odeillo-Via (66124),Pyrénées-Orientales (66),4010,27/06/2012,19/06/2023,6


In [26]:
# curl 'https://ades.eaufrance.fr/Fiche/MesureGraph?CodeBSS=10795X0001%2FS&DateDebut=27%2F02%2F2002&DateFin=11%2F01%2F2011&Couple=3&Lenght=3034'
# curl 'https://ades.eaufrance.fr/Fiche/MesureGraph?CodeBSS=10947X0084/F&DateDebut=0&DateFin=19%2F06%2F2023&Lenght=999999'

from pprint import pprint
import json

def ades_FicheMesureGraph(row):
    url = 'https://ades.eaufrance.fr/Fiche/MesureGraph'
    query_params = {
        'CodeBSS': row['Ancien code national BSS'],
        'DateDebut': 0,
        # 'DateDebut': row['Date de début'],
        'DateFin': row['Date de fin'],
        # 'Couple': row['type_pt_eau'],
        'Lenght': 10**7
        # 'Lenght': row['Nb de mesures']
    }
    response_FicheMesureGraph = requests.get(url=url, params=query_params)
    return response_FicheMesureGraph.content

pd.DataFrame(
    json.loads(
        ades_FicheMesureGraph(df_point_eau.loc[0,]).decode()
    )['ValeurReelNgf']
)


,x,y
0,1340827200000,1508.21
1,1340870400000,1508.23
2,1340960400000,1508.24
3,1341054000000,1508.25
4,1341144000000,1508.27
...,...,...
4005,1686801600000,1508.46
4006,1686942000000,1508.46
4007,1687032000000,1508.47
4008,1687122000000,1508.47


In [ ]:
curl 'https://ades.eaufrance.fr/Fiche/MesureTabDonnees' -X POST \
-H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0' \
-H 'Accept: application/json, text/javascript, */*; q=0.01' \
-H 'Accept-Language: en-US,en;q=0.7,fr-FR;q=0.3' \
-H 'Accept-Encoding: gzip, deflate, br' \
-H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
-H 'X-Requested-With: XMLHttpRequest' \
-H 'Origin: https://ades.eaufrance.fr' \
-H 'DNT: 1' \
-H 'Connection: keep-alive' \
-H 'Referer: https://ades.eaufrance.fr/Fiche/PtEau?Code=10795X0001/S' \
-H 'Cookie: ASP.NET_SessionId=5h0ea5equ1dgpv1afo1o35ex; BIGipServerpool_ades_public_https=2517346496.47873.0000; TS019555ba=01ce77d600f0ec24bf8bf34d88a0bed770976920a8752e1fde4e397e7cf6a95af60b1f2794a590960ea8f71d99724bd0e747659886cd4e82b42619bbb624df6cce836125fc70d1f8d01e84c595bc868a511e7cd854' \
-H 'Sec-Fetch-Dest: empty' \
-H 'Sec-Fetch-Mode: cors' \
-H 'Sec-Fetch-Site: same-origin' \
--data-raw 'draw=1&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=asc&start=0&length=2000&search%5Bvalue%5D=&search%5Bregex%5D=false&CodeBSS=10795X0001%2FS&DateDebut=27%2F02%2F2002&DateFin=11%2F01%2F2011&Couples=3&KnowNbResult=false'